# Imports and inits

In [32]:
import numpy as np
import sympy as sp
from sympy import cos, sin

In [40]:
t = sp.Symbol('t')
qs = [sp.Function(f'q_{i}')(t) for i in range(6)] # Joints effect
ls = sp.symbols(r"l_:6") # Link lengths
hs = sp.symbols(r"h_:3") # Link heights
ws = sp.symbols(r"w_:3") # Link widths
rs = sp.symbols(r"r_:6") # Link radii

In [41]:
qs

[q_0(t), q_1(t), q_2(t), q_3(t), q_4(t), q_5(t)]

In [42]:
# Source: symbolic_fk.ipynb demo code

def rz(theta):
    rot = sp.zeros(4,4)
    rot[3,3] = 1
    rot[0,:] = [[cos(theta), -sin(theta),0,0]]
    rot[1,:] = [[sin(theta), cos(theta),0,0]]
    rot[2,2] = 1

    return rot

def rx(theta): 
    rot = sp.zeros(4,4)
    rot[3,3] = 1
    rot[0,0] = 1
    rot[1,:] = [[0, cos(theta), -sin(theta),0]]
    rot[2,:] = [[0, sin(theta), cos(theta),0]]

    return rot


def ry(theta): 
    rot = sp.zeros(4,4)
    rot[3,3] = 1
    rot[1,1] = 1
    rot[0,:] = [[ cos(theta), 0, sin(theta),0]]
    rot[2,:] = [[-sin(theta),0, cos(theta),0]]

    return rot


def trans(vector):
    mat = sp.eye(4)
    mat[0:3,3] = vector
    return mat

def tx(dx):
    return trans([dx, 0, 0])

def ty(dy):
    return trans([0, dy, 0])

def tz(dz):
    return trans([0, 0, dz])

# Calculating transformation

In [63]:
dh_table = [
    [0, 0, 0, 0],
    [sp.pi / 2, sp.pi / 2, 0, qs[1] + ls[1]],
    [0, -sp.pi / 2, 0, qs[2] + ls[2]],
    [-sp.pi / 2 + qs[3], sp.pi / 2, 0, ls[3]],
    [sp.pi / 2 + qs[4], sp.pi / 2, 0, 0],
    [qs[5], 0, 0, ls[4]]
]

In [67]:
def dh_to_matrix(dh_row):
    theta, alpha, a, d = dh_row
    return rz(theta) * tx(a) * tz(d) * rx(alpha)

In [75]:
print('All consequent transformations:')
T = sp.eye(4)
for i in range(len(dh_table)):
    print(f'{i} -> {i + 1}')
    res = dh_to_matrix(dh_table[i])
    T = T * res
    display(res)

print('Total transformation:')
display(T)

All consequent transformations:
0 -> 1


Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

1 -> 2


Matrix([
[0, 0, 1,            0],
[1, 0, 0,            0],
[0, 1, 0, l_1 + q_1(t)],
[0, 0, 0,            1]])

2 -> 3


...

Matrix([
[-sin(q_4(t)), 0, cos(q_4(t)), 0],
[ cos(q_4(t)), 0, sin(q_4(t)), 0],
[           0, 1,           0, 0],
[           0, 0,           0, 1]])

5 -> 6


Matrix([
[cos(q_5(t)), -sin(q_5(t)), 0,   0],
[sin(q_5(t)),  cos(q_5(t)), 0,   0],
[          0,            0, 1, l_4],
[          0,            0, 0,   1]])

Total transformation:


Matrix([
[ sin(q_3(t))*sin(q_5(t)) - sin(q_4(t))*cos(q_3(t))*cos(q_5(t)), sin(q_3(t))*cos(q_5(t)) + sin(q_4(t))*sin(q_5(t))*cos(q_3(t)), cos(q_3(t))*cos(q_4(t)), l_2 + l_4*cos(q_3(t))*cos(q_4(t)) + q_2(t)],
[-sin(q_3(t))*sin(q_4(t))*cos(q_5(t)) - sin(q_5(t))*cos(q_3(t)), sin(q_3(t))*sin(q_4(t))*sin(q_5(t)) - cos(q_3(t))*cos(q_5(t)), sin(q_3(t))*cos(q_4(t)),                l_4*sin(q_3(t))*cos(q_4(t))],
[                                       cos(q_4(t))*cos(q_5(t)),                                      -sin(q_5(t))*cos(q_4(t)),             sin(q_4(t)),       l_1 + l_3 + l_4*sin(q_4(t)) + q_1(t)],
[                                                             0,                                                             0,                       0,                                          1]])

In [76]:
print('Translation:')
display(T[0:3,3])
print('Rotation:')
display(T[0:3,0:3])

Translation:


Matrix([
[l_2 + l_4*cos(q_3(t))*cos(q_4(t)) + q_2(t)],
[               l_4*sin(q_3(t))*cos(q_4(t))],
[      l_1 + l_3 + l_4*sin(q_4(t)) + q_1(t)]])

Rotation:


Matrix([
[ sin(q_3(t))*sin(q_5(t)) - sin(q_4(t))*cos(q_3(t))*cos(q_5(t)), sin(q_3(t))*cos(q_5(t)) + sin(q_4(t))*sin(q_5(t))*cos(q_3(t)), cos(q_3(t))*cos(q_4(t))],
[-sin(q_3(t))*sin(q_4(t))*cos(q_5(t)) - sin(q_5(t))*cos(q_3(t)), sin(q_3(t))*sin(q_4(t))*sin(q_5(t)) - cos(q_3(t))*cos(q_5(t)), sin(q_3(t))*cos(q_4(t))],
[                                       cos(q_4(t))*cos(q_5(t)),                                      -sin(q_5(t))*cos(q_4(t)),             sin(q_4(t))]])

In [77]:
print(sp.latex(T))

\left[\begin{matrix}\sin{\left(\operatorname{q_{3}}{\left(t \right)} \right)} \sin{\left(\operatorname{q_{5}}{\left(t \right)} \right)} - \sin{\left(\operatorname{q_{4}}{\left(t \right)} \right)} \cos{\left(\operatorname{q_{3}}{\left(t \right)} \right)} \cos{\left(\operatorname{q_{5}}{\left(t \right)} \right)} & \sin{\left(\operatorname{q_{3}}{\left(t \right)} \right)} \cos{\left(\operatorname{q_{5}}{\left(t \right)} \right)} + \sin{\left(\operatorname{q_{4}}{\left(t \right)} \right)} \sin{\left(\operatorname{q_{5}}{\left(t \right)} \right)} \cos{\left(\operatorname{q_{3}}{\left(t \right)} \right)} & \cos{\left(\operatorname{q_{3}}{\left(t \right)} \right)} \cos{\left(\operatorname{q_{4}}{\left(t \right)} \right)} & l_{2} + l_{4} \cos{\left(\operatorname{q_{3}}{\left(t \right)} \right)} \cos{\left(\operatorname{q_{4}}{\left(t \right)} \right)} + \operatorname{q_{2}}{\left(t \right)}\\- \sin{\left(\operatorname{q_{3}}{\left(t \right)} \right)} \sin{\left(\operatorname{q_{4}}{\left(t \ri